In [1]:
import pandas as pd
from stanfordcorenlp import StanfordCoreNLP
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import csv
import nltk
import statistics
import subprocess
import shlex
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import swifter
from itertools import zip_longest
from tqdm import tqdm
import numpy as np

In [2]:
df = pd.read_csv('../../data/all_cleaned.csv')

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('vader_lexicon')
s = SentimentIntensityAnalyzer()


In [4]:

nlp = StanfordCoreNLP('../../data/stanford-corenlp-full-2018-10-05/')
stops = set(stopwords.words("english"))
anew = "lib/EnglishShortened.csv"
lmtzr = WordNetLemmatizer()

#source https://github.com/dwzhou/SentimentAnalysis/blob/master/src/AnewSentimentAnalysis.py
def valence_arousal_dominance(fulltext):
    """
    Performs sentiment analysis on the text file given as input using the ANEW database.
    Outputs results to a new CSV file in output_dir.
    :param fulltext: text to analyze
    :return: v_score, a_score, d_score
    """
    sentences = tokenize.sent_tokenize(fulltext)
    i = 1
    
    total_words = 0
    v_score = 0
    a_score = 0
    d_score = 0
    found_words = []
    
    for s in sentences:
        # search for each valid word's sentiment in ANEW
        words = nlp.pos_tag(s.lower())
        for index, p in enumerate(words):
            # don't process stops or words w/ punctuation
            w = p[0]
            pos = p[1]
            if w in stops or not w.isalpha():
                continue

            # check for negation in 3 words before current word
            j = index-1
            neg = False
            while j >= 0 and j >= index-3:
                if words[j][0] == 'not' or words[j][0] == 'no' or words[j][0] == 'n\'t':
                    neg = True
                    break
                j -= 1

            # lemmatize word based on pos
            if pos[0] == 'N' or pos[0] == 'V':
                lemma = lmtzr.lemmatize(w, pos=pos[0].lower())
            else:
                lemma = w
            
            with open(anew) as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    if row['Word'].casefold() == lemma.casefold():
                        if neg:
                            found_words.append("neg-"+lemma)
                        else:
                            found_words.append(lemma)
                        v = float(row['valence'])
                        a = float(row['arousal'])
                        d = float(row['dominance'])

                        if neg:
                            # reverse polarity for this word
                            v = 5 - (v - 5)
                            a = 5 - (a - 5)
                            d = 5 - (d - 5)

                        v_score = v_score + v
                        a_score = a_score + a
                        d_score = d_score + d

    if len(found_words) != 0:  # no words found in ANEW for this sentence
        v_score = v_score/len(found_words)
        a_score = a_score/len(found_words)
        d_score = d_score/len(found_words)
            
    
    return v_score, a_score, d_score

In [5]:
#too slow don't do this
# val_a_d = df['Tweet'].progress_apply(valence_arousal_dominance)

In [31]:
#can make this much efficient
def perform_sentiment_analysis(x):
    x = str(x)
    vader_emotion = s.polarity_scores(x)['compound']
    sen = TextBlob(x).sentiment
    polarity = sen.polarity
    subjectivity = sen.subjectivity
    
    return vader_emotion, polarity, subjectivity

In [9]:
emotions = df['Tweet'].swifter.apply(perform_sentiment_analysis)

Pandas Apply: 100%|██████████| 7039259/7039259 [2:10:11<00:00, 901.18it/s]


In [33]:
emotions_df=pd.DataFrame(emotions.tolist(), columns=['vader_emotion', 'polarity', 'subjectivity'])

In [5]:
df = df.join(emotions_df)

In [22]:
# df['Tweet'] = df['Tweet'].fillna("NA")

# df['Tweet'] = df['Tweet'].replace(r'\\n',' ', regex=True) 
# df['Tweet'] = df['Tweet'].replace(r'\n',' ', regex=True) 

# df['Tweet'] = df['Tweet'].replace(r'\\r',' ', regex=True) 
# df['Tweet'] = df['Tweet'].replace(r'\r',' ', regex=True) 

# df['Tweet'] = df['Tweet'].replace(r'\\t',' ', regex=True) 
# df['Tweet'] = df['Tweet'].replace(r'\t',' ', regex=True) 

# df['Tweet'].to_csv('tweets', index=None, header=None)

# java -jar SentiStrength.jar sentidata SentiStrength_Data/ input tweets

In [6]:
aa = pd.read_csv('tweets0_out.txt', sep="\t")

In [9]:
df = df.join(aa[['Positive', 'Negative']])

In [12]:
df.to_csv('../../data/all_cleaned.csv', index=None)